## Algoritmia
### Práctica 10
En esta práctica se consideran problemas sobre grafos que son NP completos: cubrimiento y clique

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Grafos
Las funciones a implementar trabajan con grafos. En vez de utilizar alguna implementación de un tipo específico para grafos, estos se representan como diccionarios donde las claves son los nodos del grafo y los valores asociados a una clave son el conjunto de nodos destino de los arcos con origen en el nodo clave.

### Clique

In [93]:
def es_clique(grafo, nodos):
    """
    Devuelve un booleano indicando si el conjunto de nodos es un clique.
    Un conjunto de nodos es un clique si hay un arco entre todos los pares de
    nodos del conjunto.
    """
    resultado = False
    vecinos = []
    lista = []
    
    for i in nodos:
        if i in grafo.keys():
            vecinos = grafo.get(i)
            lista = list(vecinos) + list(nodos)
            for j in lista:
                if i != j:
                    if j in vecinos:
                        resultado = True
                    else:
                        return False
        else:
            return False
    return resultado
    

### Cubrimiento

In [94]:
def es_cubrimiento(grafo, nodos):
    """
    Devuelve un booleano indicando si los nodos son un cubrimiento del grafo.
    Un conjunto de nodos es un cubrimiento si todos los arcos son adyacentes a
    al menos un nodo del conjunto.
    """
    resultado = False
    vecinos = []
    
    for i in grafo.keys():
        contador = 0
        vecinos = grafo.get(i)
        if i in nodos:
            resultado = True
        else:
            for j in vecinos:
                if not j in nodos:
                    return False
                else:
                    contador=contador+1
                    if contador == len(vecinos):
                        resultado = True
    return resultado

### Grafos de prueba

In [95]:
def grafo_de_prueba(num_nodos):
    """
    Función que crea un grafo de prueba. Recibe un número de nodos, el grafo
    generado tendrá el doble de nodos, en dos grupos: a0, a1, a2... y b0, b1, 
    b2...
    Hay un arco entre cada par de nodos con el mismo índice: a0-b0, a1-b1...
    Hay un arco entre todos los nodos a0, a1, a2... 
    """
    
    grafo = {}
    for i in range(num_nodos):
        nodo_a = "a" + str(i)
        nodo_b = "b" + str(i)
        grafo[nodo_a] = {nodo_b}
        grafo[nodo_b] = {nodo_a}
        for j in range(i):
            n = "a" + str(j)
            grafo[nodo_a].add(n)
            grafo[n].add(nodo_a)
    return grafo

### Tests para `es_clique`

In [96]:
def test_es_clique(num_nodos):
    """Función que prueba es_clique."""

    grafo = grafo_de_prueba(num_nodos)
    nodos_a, nodos_b, nodos = set(), set(), set()
    for i in range(num_nodos):
        a = "a" + str(i)
        b = "b" + str(i)
        nodos_a.add(a)
        nodos_b.add(b)
        nodos.update((a,b))
        assert es_clique(grafo, nodos_a)
        if i > 0:
            assert not es_clique(grafo, nodos_b)
            assert not es_clique(grafo, nodos)   
            
if __name__ == "__main__": 
    test_es_clique(10)
    print("OK")               

OK


### Tests para `es_cubrimiento`

In [97]:
def test_es_cubrimiento(num_nodos):
    """Función que prueba es_cubrimiento."""

    grafo = grafo_de_prueba(num_nodos)
    nodos_a, nodos_b, nodos = set(), set(), set()
    for i in range(num_nodos):
        assert not es_cubrimiento(grafo, nodos_a)
        assert not es_cubrimiento(grafo, nodos_b)
        assert not es_cubrimiento(grafo, nodos)
        a = "a" + str(i)
        b = "b" + str(i)
        nodos_a.add(a)
        nodos_b.add(b)
        nodos.update((a,b))
    assert es_cubrimiento(grafo, nodos_a)
    assert not es_cubrimiento(grafo, nodos_b)
    assert es_cubrimiento(grafo, nodos)
    
if __name__ == "__main__": 
    test_es_cubrimiento(10)
    print("OK")    

OK


### Clique máximo

In [102]:
import itertools

def clique_maximo(grafo):
    """Dado un grafo, devuelve un conjunto de nodos que es un clique máximo."""
    s=grafo.keys()
    resultado=[]
    test=False
    
    def subsets(s):
        combinaciones = []
        for i in range(1,len(grafo)+1):
            combinaciones += itertools.combinations(s,i)
        return combinaciones
    
    sets = subsets(s)
    sets.reverse()
    for i in sets:
        if es_clique(grafo,i)==True:
            for j in list(i):
                resultado.append(j)
            test=True
            return set(resultado)
        
    if test==False:
        return s

### Tests para `clique_maximo`

In [103]:
def test_clique_maximo(max_nodos):
    """Función que prueba clique_maximo."""    
            
    # Grafo lineales, cada nodo está conectado solo con el siguiente:
    grafo = {1 : set()}
    for i in range(2, max_nodos + 1):
        grafo[i - 1].add(i)
        grafo[i] = {i - 1}
        clique = clique_maximo(grafo)
        assert es_clique(grafo, clique)
        assert len(clique) == 2

    # Grafos completos, cada nodo está conectado con todos los demás:
    grafo = {1 : set()}
    for i in range(2, max_nodos + 1):
        grafo[i] = {j for j in range(1,i)}
        for j in range(1, i):
            grafo[j].add(i)
        clique = clique_maximo(grafo)
        assert es_clique(grafo, clique)
        assert len(clique) == i

    # Grafos generados con grafo_de_prueba()
    for i in range(3, max_nodos + 1):
        grafo = grafo_de_prueba(i)
        clique = clique_maximo(grafo)
        assert len(clique) == i
        assert es_clique(grafo, clique)
        clique.add("b0")
        assert not es_clique(grafo, clique)
            
if __name__ == "__main__": 
    test_clique_maximo(10)
    print("OK")   

OK


### Cubrimiento mínimo

In [89]:
import itertools

def cubrimiento_minimo(grafo):
    """
    Dado un grafo, devuelve un conjunto de nodos que es un cubrimiento 
    mínimo.
    """
    s=grafo.keys()
    resultado=[]
    test=False
    
    def subsets(s):
        combinaciones = []
        for i in range(1,len(grafo)+1):
            combinaciones += itertools.combinations(s,i)
        return combinaciones
    
    sets = subsets(s)
    for i in sets:
        if es_cubrimiento(grafo,i)==True:
            for j in list(i):
                resultado.append(j)
            test=True
            return resultado
        
    if test==False:
        return s
        

### Tests para `cubrimiento_minimo`

In [90]:
def test_cubrimiento_minimo(max_nodos):
    """Función que prueba cubrimiento_minimo."""    
           
    # Grafos lineales, cada nodo está conectado solo con el siguiente:
    grafo = {1 : set()}
    for i in range(2, max_nodos + 1):
        grafo[i - 1].add(i)
        grafo[i] = {i - 1}
        cubrimiento = cubrimiento_minimo(grafo)
        assert es_cubrimiento(grafo, cubrimiento)
        assert len(cubrimiento) == i // 2
        cubrimiento.pop()
        assert not es_cubrimiento(grafo, cubrimiento)

    # Grafos completos, cada nodo está conectado con todos los demás:
    grafo = {1 : set()}
    for i in range(2, max_nodos + 1):
        grafo[i] = {j for j in range(1,i)}
        for j in range(1, i):
            grafo[j].add(i)
        cubrimiento = cubrimiento_minimo(grafo)
        assert es_cubrimiento(grafo, cubrimiento)
        assert len(cubrimiento) == i - 1
        cubrimiento.pop()
        assert not es_cubrimiento(grafo, cubrimiento)
        
    # Grafos generados con grafo_de_prueba()
    for i in range(1, max_nodos + 1):
        grafo = grafo_de_prueba(i)
        cubrimiento = cubrimiento_minimo(grafo)
        assert len(cubrimiento) == i
        assert es_cubrimiento(grafo, cubrimiento)   
        cubrimiento.pop()
        assert not es_cubrimiento(grafo, cubrimiento)
            
if __name__ == "__main__": 
    test_cubrimiento_minimo(10)
    print("OK")   

OK
